In [113]:
from datetime import timedelta
import cv2
import numpy as np
import os

In [114]:
taken_fps = 15

In [115]:
def taken_frames_durations(cap, saving_fps):
    duration = []
    video_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    for i in np.arange(0, video_duration, 1 / saving_fps):
        duration.append(round(i,2))
    return duration

In [116]:
store_image = list()
video_directory = "F:\\Research database\\Avenue Dataset\\training_videos"
frames_directory = "F:\\Research database\\Avenue Dataset\\training_frames"


## Frame extraction 

In [117]:
video_names = os.listdir(video_directory) 
for name in video_names:
    saving_dir_name, _ = os.path.splitext(name)
    saving_dir_name = "video_" + saving_dir_name
    video_path = os.path.join(video_directory, name)
    saving_frame_path = os.path.join(frames_directory,saving_dir_name)
    if not os.path.isdir(saving_frame_path):
        os.mkdir(saving_frame_path)
    cap = cv2.VideoCapture(video_path) 
    fps = cap.get(cv2.CAP_PROP_FPS)
    final_taken_fps = min(fps, taken_fps) # taken_fps should not higher than video fps
    saving_frames_durations = taken_frames_durations(cap, final_taken_fps)
    time_count = 0
    frame_number = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            break
        frame_duration = time_count / fps # measure duration until video end
        try:
            closest_duration = saving_frames_durations[0]
        except:
            break
        if frame_duration >= closest_duration: #this condition maintain taken fps
            frame = cv2.resize(frame,(227,227))
            frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            cv2.imwrite(os.path.join(saving_frame_path, f"frame_{frame_number}.jpg"),frame) #save frame for backup
            store_image.append(frame)
            frame_number += 1
            try:
                saving_frames_durations.pop(0) #as 0th postion recored so I removed 0th frame
            except:
                pass
        time_count += 1

## Dumping all frames as numpy file

In [118]:
store_image=np.array(store_image)

In [119]:
# normalization
store_image=(store_image-store_image.mean())/(store_image.std())
store_image=np.clip(store_image,0,1) # make between 0 to 1

In [120]:
np.save('training.npy',store_image)